# Computational time for alexnet / resnet

In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd


def build_dataset(p: Path):
    dfs = []
    for folder in p.glob('**'):
        for file in folder.glob('*.csv'):
            dfs.append(pd.read_csv(file))
            dfs[-1]['Seed'] = int(re.findall(r'\d+', file.name)[0])
            dfs[-1]['Problem'] = file.parent.parent.name

    df = pd.concat(dfs)
    df['Loglr'] = df['Learning rate'].apply(lambda x: np.log10(x))

    model_dict = {
        "model_type.alexnet" : "Alexnet",
        "model_type.halexnet" : "H-Alexnet",
        "model_type.hresnet18" : "H-ResNet18",
        "model_type.resnet18" : "ResNet18"
    }

    df['Model'] = df['Model'].apply(lambda x: model_dict[x])

    return df

In [2]:
df = build_dataset(Path('../data/halexnet/'))

In [3]:
df1 = df[['Model', 'Seed', 'Problem', 'Time']].groupby(['Model', 'Seed', 'Problem'], as_index=False).apply(np.maximum.reduce)

/tmp/ipykernel_1418662/3212935086.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df1 = df[['Model', 'Seed', 'Problem', 'Time']].groupby(['Model', 'Seed', 'Problem'], as_index=False).apply(np.maximum.reduce)


In [4]:
import datetime

def seconds_to_hms_padded(s):
    total = int(s)
    h, rem = divmod(total, 3600)
    m, sec = divmod(rem, 60)
    return f"{h}:{m:02d}:{sec:02d}"

times = df1[['Model', 'Time']].groupby(['Model'], as_index=False).apply(np.sum, include_groups=False, axis=0)
times['Time']=times['Time'].apply(seconds_to_hms_padded)
times

,Model,Time
0,Alexnet,43:21:01
1,H-Alexnet,69:24:27


In [5]:
156000/3600

43.333333333333336

In [6]:
resnet=build_dataset(Path('../data/all_conv_aug'))
df1 = resnet[['Model', 'Seed', 'Problem', 'Time']].groupby(['Model', 'Seed', 'Problem'], as_index=False).apply(np.maximum.reduce)

/tmp/ipykernel_1418662/258945074.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df1 = resnet[['Model', 'Seed', 'Problem', 'Time']].groupby(['Model', 'Seed', 'Problem'], as_index=False).apply(np.maximum.reduce)


In [7]:
import datetime

def seconds_to_hms_padded(s):
    total = int(s)
    h, rem = divmod(total, 3600)
    m, sec = divmod(rem, 60)
    return f"{h}:{m:02d}:{sec:02d}"

times = df1[['Model', 'Time']].groupby(['Model'], as_index=False).apply(np.sum, include_groups=False, axis=0)
times['Time']=times['Time'].apply(seconds_to_hms_padded)
times

,Model,Time
0,H-ResNet18,101:36:00
1,ResNet18,52:46:40
